![Ejemplo 3](images/Ej3.png)

En este ejercicio utilizaremos la notación compacta general para cualquier número de intervalo y corrientes calientes y frías. Para ello, tendremos que leer de dos archivos Excel los aportes de las corrientes calientes y la eliminación de las frías. Estos datos, en forma de tabla, están en el archivo "Ejercicio3.xlsx", en las hojas "Hot" y "Cold", respectivamente.

En este caso, tendremos que usar la librería Pandas, con la que importaremos los datos para, posteriormente usarlos dentro del modelo de optimización.

In [25]:
from pyomo.environ import *
import pandas as pd

model = ConcreteModel()

Leemos los datos de las corrientes calientes

In [26]:
Hot_df = pd.read_excel('Datos/Ejercicio3.xlsx', 'Hot',skiprows = 1,index_col=0)
Hot_df


,0,1,2,3,4,5,6,7,8,9
ID,,,,,,,,,,
H1,0,360,60,180,60,180,60,0,0,0
H2,0,0,0,0,0,540,0,0,0,0
H3,0,0,0,240,80,240,80,160,80,160


E igualmente de las frías.

In [27]:
Cold_df = pd.read_excel('Datos/Ejercicio3.xlsx', 'NewCold3',skiprows = 1, index_col=0)
Cold_df

,0,1,2,3,4,5,6,7,8,9
ID,,,,,,,,,,
C1,0,0,150,450,150,450,150,300,150,0
C2,0,0,0,0,160,480,160,320,0,0
C3,0,0,0,0,0,0,0,400,200,400


Las columnas de las tablas anteriores son los diferentes intervalos de temperatura (9+1 ya que incluimos un intervalo 0 que necesitamos porque en el balance aparece el término R(i-1)), mientras que las filas son las corrientes calientes y frías respectivamente. Con estos datos crearemos tres índices para el modelo: uno (h) contendrá los identificadores de las corrientes calientes (H1, H2 y H3), otro (c) de las frías (C1 y C2) y un último (i) con los diez intervalos (0-9)

In [28]:
hotst = Hot_df.index.values.tolist()
coldst = Cold_df.index.values.tolist()
model.h=Set(initialize=hotst)
model.c=Set(initialize=coldst)
model.i=Set(initialize=Hot_df.columns)

Podemos ver lo que hemos creado hasta ahora haciendo un model.pprint()

In [29]:
model.pprint()

3 Set Declarations
    c : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'C1', 'C2', 'C3'}
    h : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'H1', 'H2', 'H3'}
    i : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

3 Declarations: h c i


Creamos ahora las variables del modelo. Hay i potenciales aportes de calor por utilities calientes (Qs), uno en cada intervalo, al igual que de frías (Qw). De esta forma damos la oportunidad de que entren las utilities en cualquier nivel de temperatura. Creamos también i residuos (R), aunque sabemos que no habrá residuo del intervalo 0 ni tampoco del intervalo 9, lo cual implementaremos como dos restricciones diferentes.

In [30]:
Qs = model.Qs = Var(model.i,within = NonNegativeReals)
Qw = model.Qc = Var(model.i,within = NonNegativeReals)
R = model.R = Var(model.i,within = NonNegativeReals)

Buscamos la minimización del vapor del intervalo 1 más la utility fría del intervalo 9.

In [31]:
model.util = Objective(expr = model.Qs[1] + model.Qc[9])

Constraints

In [32]:
ni = list(model.i)[1:] #Creamos una lista que contenga los índices sobre los que haremos el balance. En nuestro caso el balance lo aplicamos en todos los intervalos, excepto el intervalo 0. Es decir sobre 1-9, lo cual expresamos por [1:], es decir, desde 1 hasta el final de la lista.
model.int = ConstraintList()
for i in ni:
    model.int.add(
        R[i-1]+Qs[i]+sum(Hot_df[i]) == R[i]+sum(Cold_df[i])+Qw[i]
    )

model.R0 = Constraint(expr = R[0] == 0) #No hay residuo ni desde intervalo 0 ni desde 9
model.R9 = Constraint(expr = R[9] == 0)

nii = list(model.i)[0:1]+list(model.i)[2:] #Con esta restricción buscamos que solo se aporte utility en el primer intervalo (no más abajo), por lo que decimos que desde 2 a 9 la variable Qs sea 0 y también que Qs[0] sea 0. Es decir, que solo dejamos como variable Qs[1]
model.steam = ConstraintList()
for i in nii:
    model.steam.add(
        Qs[i]==0
    )

niii = list(model.i)[0:9] #Lo mismo deantes para la utility fría. Es decir de 0 a 8 (el 9 no está incluido en el intervalo al expresarlo así en Python) Qw es 0.
model.cw = ConstraintList()
for i in niii:
    model.cw.add(
        Qw[i]==0
    )



Resolvemos el modelo

In [33]:
results = SolverFactory('glpk').solve(model)
model.pprint()
results.write()

6 Set Declarations
    c : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'C1', 'C2', 'C3'}
    cw_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    9 : {1, 2, 3, 4, 5, 6, 7, 8, 9}
    h : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'H1', 'H2', 'H3'}
    i : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
    int_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    9 : {1, 2, 3, 4, 5, 6, 7, 8, 9}
    steam_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    9 : {1, 2, 3, 4, 5, 6, 7, 8, 9}

3 Var Decl

In [34]:
Qh = value(model.Qs[1])
Qc = value(model.Qc[9])
print('Cold utility = {0:2.2f}, Hot utility = {1:2.2f}'.format(Qc, Qh))

Cold utility = 0.00, Hot utility = 1440.00
